In [1]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

import onnx
import onnxruntime
from nix.models.TTS import NixTTSInference
from IPython.display import Audio
import soundfile as sf

import toybox

Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory


## [seq] device setting

In [2]:
import os
#print(os.environ['OMP_NUM_THREADS'])
print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.

all cpu at using device: 52
Number of available CPU: 4


In [3]:
DEVICE = 'cpu' # 'cuda' or 'cpu'
#DEVICE_UTMOS = 'cuda'
#if torch.cuda.is_available():
#    print('use cuda')
#else:
#    #os._exit(os.EX_OK)
#    print('use cpu')

device = torch.device(DEVICE)
print(f'device: {device}')

device: cpu


In [4]:
# seed val from https://github.com/sarulab-speech/UTMOS22/blob/master/strong/configs/train/default.yaml
random_seed = 1234

n_mels: int = 80
n_fft: int = 1024
sample_rate: int = 22050
hop_size: int = 256
win_size: int = 1024
f_min: int = 0
f_max: int = 8000

toybox.set_seed(random_seed)

print(f"phn2id_path: ./configs/phn2id.json")
with open("./configs/phn2id.json") as f:
    phn2id = json.load(f)

vocab_size = len(phn2id) + 1

onnxruntime.set_seed(random_seed)
toybox.set_seed(random_seed)

phn2id_path: ./configs/phn2id.json


## [seq] setting path

In [6]:
test_ds_path = Path('configs/test_dataset.json') #Path(config['test_datalist_path'])

model_name = 'nix_deter'#'deter' or 'stoch'
runtime_name = 'infer4colb'
hifigan_versions = 'LJ_V1'
wav_dir_name = f'wav_{hifigan_versions}'

if model_name=='nix_stoch':
    ckpt_path = Path('./nix/pretrained/nix-ljspeech-stochastic-v0.1')
elif model_name=='nix_deter':
    ckpt_path = Path('./nix/pretrained/nix-ljspeech-deterministic-v0.1')
else:
    print("Not found")

if ckpt_path.exists()==True:
    print(f"ckpt_path: {ckpt_path}")
else:
    print(f"Not find")


RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/e500_n50')
#results_dir_path = Path(f'./result4nixeval/{model_type}')

#RESULT_MEL_DIR_PATH = RESULT_DIR_PATH / 'mel_LJ_V1'
RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / wav_dir_name
#wav_dir_path = results_dir_path / 'nix_wavs'
RESULT_JSON_PATH = RESULT_DIR_PATH / f'eval4mid_{hifigan_versions}.json'

# for text2mel
print('test_ds_path-----------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(test_ds_path) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('RESULT_DIR_PATH-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_DIR_PATH}')

print('RESULT_WAV_DIR_PATH-------------------------------------------')
if RESULT_WAV_DIR_PATH.exists():
    print(f'Exists {RESULT_WAV_DIR_PATH}')
else:
    RESULT_WAV_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_WAV_DIR_PATH}')

print('RESULT_JSON_PATH-------------------------------------------')
if RESULT_JSON_PATH.exists():
    print(f'Exists {RESULT_JSON_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_JSON_PATH}')

"""
if eval_json_path.exists()==True:
    with open(eval_json_path) as f:
        info_jsonl = [json.loads(l) for l in f.readlines()]
else:
    print('No {eval_json_path}')
"""

ckpt_path: nix/pretrained/nix-ljspeech-deterministic-v0.1
test_ds_path-----------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
RESULT_DIR_PATH-------------------------------------------
No exist result4eval/infer4colb/nix_deter/cpu/e500_n50
RESULT_WAV_DIR_PATH-------------------------------------------
No exist result4eval/infer4colb/nix_deter/cpu/e500_n50/wav_LJ_V1
RESULT_JSON_PATH-------------------------------------------
No exist result4eval/infer4colb/nix_deter/cpu/e500_n50/eval4mid_LJ_V1.json


"\nif eval_json_path.exists()==True:\n    with open(eval_json_path) as f:\n        info_jsonl = [json.loads(l) for l in f.readlines()]\nelse:\n    print('No {eval_json_path}')\n"

## [seq] load model

In [7]:
print("[seq] loading Model")

print("loading NIX ===================================")
nix = NixTTSInference(model_dir = str(ckpt_path))

print("loading UTMOS ===================================")
predictor_utmosv1 = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

[seq] loading Model
loading NIX ===================================
loading UTMOS ===================================


Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## test_run

In [8]:
input_text = test_ds_list[3]['text']
#texts = text.split('. ')
c, c_length, phonemes = nix.tokenize(input_text)
phonemes_len_int = len(phonemes[0])
phnid_len_int = c_length.item()
print(phonemes_len_int)
print(phnid_len_int)
# [seq] synth speech
audio = nix.vocalize(c, c_length)

test_wav_path = "./gomi.wav"
write(
    test_wav_path,
    sample_rate,
    audio[0,0])

print(test_wav_path)
test_waveform, samplerate = torchaudio.load(test_wav_path)
utmosv1_score = predictor_utmosv1(test_waveform, samplerate)
print(f'utmosv1_score: {utmosv1_score.item()}')
ipd.display(ipd.Audio(test_waveform, rate=samplerate))

['The rifle without the scope cost only twelve dollars, seventy-eight cents.']
78
157
./gomi.wav
utmosv1_score: 3.7904062271118164


## [seq] eval

In [9]:
infer_data_num: int = 101 #len(test_ds_list) is 200
print(f'infer_data_num: {infer_data_num}')
print(f'RESULT_DIR_PATH: {RESULT_DIR_PATH}')
print(f'RESULT_MEL_DIR_PATH: {RESULT_DIR_PATH}')
print(f'RESULT_WAV_DIR_PATH: {RESULT_WAV_DIR_PATH}')
print(f'RESULT_JSON_PATH: {RESULT_JSON_PATH}')

infer_data_num: 101
RESULT_DIR_PATH: result4eval/infer4colb/nix_deter/cpu/e500_n50
RESULT_MEL_DIR_PATH: result4eval/infer4colb/nix_deter/cpu/e500_n50
RESULT_WAV_DIR_PATH: result4eval/infer4colb/nix_deter/cpu/e500_n50/wav_LJ_V1
RESULT_JSON_PATH: result4eval/infer4colb/nix_deter/cpu/e500_n50/eval4mid_LJ_V1.json


In [10]:
eval_list = []

for i in tqdm(range(infer_data_num)):
    test_ds_filename = test_ds_list[i]['name']
    synth_wav_path = RESULT_WAV_DIR_PATH / f"{test_ds_filename}.wav"
    print(f'test_ds_index_{i}: {test_ds_filename}')
    # [seq]text2wav =========================================================
    # load txt
    print('[seq]text2wav')
    text = test_ds_list[i]['text']
    c, c_length, phonemes = nix.tokenize(text)
    #phonemes_len_int = len(phonemes[0])
    #phnid_len_int = len(c_length[0])
    phonemes_len_int = len(phonemes[0])
    phnid_len_int = c_length.item()
    print(f'phonemes_len: {phonemes_len_int}')
    print(f'phnid_len: {phnid_len_int}')
    # [seq] synth speech
    start_time = time.perf_counter()

    audio = nix.vocalize(c, c_length)

    end_time = time.perf_counter()

    dt4wav = end_time - start_time
    audio_playback_time = (len(audio[0,0]) / 22050)
    RTF4wav = dt4wav / audio_playback_time
    print(f'dt[s]: {dt4wav}')
    print(f'RTF: {RTF4wav}')

    write(
        synth_wav_path,
        sample_rate,
        audio[0,0])

    # [seq]wav2utmos =========================================================
    print('[seq]wav2utmos')
    waveform, samplerate = torchaudio.load(synth_wav_path)
    utmosv1_score = predictor_utmosv1(waveform, samplerate)
    utmosv1_score_float = utmosv1_score.item()
    print(f'utmosv1: {utmosv1_score_float}')
    
    eval_dict = {
        'name': test_ds_filename,
        'wav_path': str(synth_wav_path),
        'phonemes_len': phonemes_len_int,
        'phnid_len': phnid_len_int,
        'audio_playback_time': audio_playback_time,
        'dt': dt4wav,
        'RTF4wav': RTF4wav,
        'utmos': utmosv1_score_float
    }
    eval_list.append(eval_dict)



  0%|                                                                                           | 0/101 [00:00<?, ?it/s]

test_ds_index_0: LJ045-0049
[seq]text2wav
['Lee would say that it would be better for me if I went to Russia. I did not know why. I did not know what he had in mind.']
phonemes_len: 131
phnid_len: 263
dt[s]: 0.9335270579904318
RTF: 0.12563642351494764
[seq]wav2utmos


  1%|▊                                                                                  | 1/101 [00:01<02:20,  1.40s/it]

utmosv1: 4.231616497039795
test_ds_index_1: LJ017-0027
[seq]text2wav
['that arsenic was within her reach in the house,']
phonemes_len: 46
phnid_len: 93


  2%|█▋                                                                                 | 2/101 [00:01<01:25,  1.16it/s]

...

utmosv1: 3.95918869972229
test_ds_index_99: LJ029-0081
[seq]text2wav
['The third possibility, the Trade Mart, a handsome new building with all the necessary facilities, presented security problems. It had numerous entrances,']
phonemes_len: 163
phnid_len: 327
dt[s]: 1.2600683495402336
RTF: 0.12153777255110124
[seq]wav2utmos


 99%|████████████████████████████████████████████████████████████████████████████████▏| 100/101 [01:39<00:01,  1.37s/it]

utmosv1: 3.918611526489258
test_ds_index_100: LJ017-0230
[seq]text2wav
["came out of the cabin, was caught near the 'companion' by the rest of the mutineers, and promptly dispatched with daggers."]
phonemes_len: 121
phnid_len: 243
dt[s]: 0.8575560543686152
RTF: 0.12229091860790023
[seq]wav2utmos


100%|█████████████████████████████████████████████████████████████████████████████████| 101/101 [01:41<00:00,  1.00s/it]

utmosv1: 4.0134782791137695


In [11]:
#RESULT_JSON_PATH = RESULT_DIR_PATH / 'eval4mid.json'
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Make result4eval/infer4colb/nix_deter/cpu/e500_n50/eval4mid_LJ_V1.json


## recheck

In [12]:
a = toybox.load_json(f'result4eval/infer4colb/{model_name}/cpu/e500_n50/eval4mid_{hifigan_versions}.json')

Exist result4eval/infer4colb/nix_deter/cpu/e500_n50/eval4mid_LJ_V1.json


In [13]:
dt_list = [a[n]['dt'] for n in range(len(a))]
RTF4mel_list = [a[n]['RTF4wav'] for n in range(len(a))]
utmos_list = [a[n]['utmos'] for n in range(len(a))]

dt_nparr = np.array(dt_list[1:101])
RTF4mel_nparr = np.array(RTF4mel_list[1:101])
utmos_nparr = np.array(utmos_list[1:101])
print(len(dt_nparr))

significant_digits = 8

# for culc difference time to infer text2mel
dt_mean = toybox.round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = toybox.round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = toybox.round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4mel_mean = toybox.round_significant_digits(np.mean(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_var = toybox.round_significant_digits(np.var(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_std = toybox.round_significant_digits(np.std(RTF4mel_nparr), significant_digits=significant_digits)
print(f'RTF ---------------------------')
print(f'RTF mean: {RTF4mel_mean}')
print(f'RTF var: {RTF4mel_var}')
print(f'RTF std: {RTF4mel_std}')

# for culc utmos to infer 
print(f'utmos ---------------------------')
utmos_mean = toybox.round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = toybox.round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = toybox.round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

100
dt ---------------------------
dt mean: 0.67199343
dt var: 0.05393388599999999
dt std: 0.2322367
RTF ---------------------------
RTF mean: 0.11887888999999999
RTF var: 2.7823638999999997e-06
RTF std: 0.0016680419
utmos ---------------------------
utmos mean: 4.041794299999999
utmos var: 0.072824227
utmos std: 0.26985964


## No process

In [44]:
eval_jsonl_path = results_dir_path / f'eval4nix_utmosv1_{model_type}.json'
print(f'eval_jsonl_path: {eval_jsonl_path}')

if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

eval_jsonl_path: result4nixeval/stoch/eval4nix_utmosv1_stoch.json
Exist result4nixeval/stoch/eval4nix_utmosv1_stoch.json


In [45]:
diff_result = list(filter(lambda x: x not in eval_list, eval_jsonl_list))
print(diff_result)

[]


## check and culc Indicator

In [17]:
model_type = 'stoch' #'deter' or 'stoch'
results_dir_path = Path(f'./result4nixeval/{model_type}')
wav_dir_path = results_dir_path / 'nix_wavs'
eval_json_path = results_dir_path / f'eval4nix_utmosv1_{model_type}.json'
print(eval_json_path)

if wav_dir_path.exists()==True:
    print(f'Already Exists {wav_dir_path}')
else:
    print(f'No  {wav_dir_path}')    

if results_dir_path.exists()==True:
    print(f'Already Exists {results_dir_path}')
else:
    print(f'No {results_dir_path}')

if eval_json_path.exists()==True:
    with open(eval_json_path) as f:
        eval_utmosv1_json = [json.loads(l) for l in f.readlines()]
else:
    print('No {eval_json_path}')

result4nixeval/stoch/eval4nix_utmosv1_stoch.json
Already Exists result4nixeval/stoch/nix_wavs
Already Exists result4nixeval/stoch


In [18]:
#diff_result = list(filter(lambda x: x not in eval_jsonl_list, info_jsonl1))
#print(diff_result)

In [19]:
def round_significant_digits(value, significant_digits=5):
    if value == 0:
        return 0

    import math
    scale = math.floor(-math.log10(abs(value)))  # Find the first nonzero after the decimal point
    factor = 10 ** (scale + significant_digits - 1)  # Scale to hold 5 significant digits

    rounded_value = round(value * factor,1) / factor  # Adjust and round off the scale
    return rounded_value

In [20]:
significant_digits = 5

utmos_list = [eval_utmosv1_json[n]['utmos'] for n in range(len(eval_utmosv1_json))]
utmos_nparr = np.array(utmos_list[1:101])
print(utmos_nparr.shape)
utmos_mean = round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

(100,)
utmos mean: 4.0298
utmos var: 0.082376
utmos std: 0.28701


In [21]:
"""
utmos mean: 4.0298
utmos var: 0.082376
utmos std: 0.28701
"""

'\nutmos mean: 4.0298\nutmos var: 0.082376\nutmos std: 0.28701\n'